In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from BendLength import BendLengthCalculator
from preprocess import preProcessData
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
point_cloud_location = "/home/chris/Code/PointClouds/data/FLIPscans/Bendy/Bendy_1/scan1_Part1.ply"
pcd = o3d.io.read_point_cloud(point_cloud_location)

mesh = o3d.io.read_triangle_mesh("/home/chris/Code/PointClouds/data/FLIPscans/Bendy/BendyCAD.STL")
mesh.compute_vertex_normals()

TriangleMesh with 4156 points and 2104 triangles.

In [3]:
def create_normal_arrow_mesh(start_point, direction, length=10.0, cylinder_radius=0.2, cone_radius=0.4, color=[0.1, 0.7, 0.3]):
    direction = direction / np.linalg.norm(direction)
    end_point = start_point + direction * length

    # Cylinder
    cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=cylinder_radius, height=length * 0.8)
    cylinder.paint_uniform_color(color)

    z_axis = np.array([0, 0, 1])
    rot_axis = np.cross(z_axis, direction)
    if np.linalg.norm(rot_axis) > 1e-6:
        rot_axis /= np.linalg.norm(rot_axis)
        angle = np.arccos(np.clip(np.dot(z_axis, direction), -1.0, 1.0))
        R = o3d.geometry.get_rotation_matrix_from_axis_angle(rot_axis * angle)
        cylinder.rotate(R, center=np.zeros(3))

    cylinder.translate(start_point + direction * (length * 0.4))

    # Cone
    cone_height = length * 0.2
    cone = o3d.geometry.TriangleMesh.create_cone(radius=cone_radius, height=cone_height)
    cone.paint_uniform_color(color)
    cone.rotate(R, center=np.zeros(3))
    cone.translate(end_point - direction * (cone_height * 1))

    return [cylinder, cone]

def draw_normal_arrows_with_geometry(segment_models, segments, main_surface_idx=None, random_flip=True):
    arrows = []

    for idx, model in segment_models.items():
        normal = np.array(model[:3])
        normal = normal / np.linalg.norm(normal)

        if random_flip and np.random.rand() > 0.5:
            normal *= -1  # Flip 180 degrees

        center = np.array(segments[idx].get_center())
        arrow_parts = create_normal_arrow_mesh(center, normal, length=8.0)
        arrows.extend(arrow_parts)

    o3d.visualization.draw_geometries(list(segments.values()) + arrows)

In [4]:
# Preprocess the point cloud
pcd, average_density, cad_pcd = preProcessData(pcd, mesh, x_rotation=90, y_rotation=20, z_rotation=0)
pcd_points = np.asarray(pcd.points)
pcd_tree = o3d.geometry.KDTreeFlann(pcd)
# Detect planes, intersections, and anchor points
segment_models, segments, segment_indices, main_surface_idx = util.multiOrderRansacAdvanced(pcd, pt_to_plane_dist=0.62, verbose=False, visualize=False)
intersection_lines = util.findIntersectionLinesLeastSquares(segment_models, main_surface_idx)
segment_models = util.flip_normals_by_bend_orientation(segment_models, intersection_lines, segments, main_surface_idx, pcd_tree, search_radius=0.8, verbose=False)

In [5]:
draw_normal_arrows_with_geometry(segment_models, segments, main_surface_idx, random_flip=False)